In [1]:
import csv
import argparse
import pandas as pd
import numpy as np
import re
import string
import os
import statistics
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.naive_bayes import GaussianNB
    
TRAIN = '../data/train/training_data.csv'
train_data = pd.read_csv(TRAIN, index_col=1)
dev_data = pd.read_csv('../data/dev/development_data.csv', index_col=1)

tweets = train_data[['tweet']]

In [2]:
bad_words_set = set(open("bad-words.txt").read().split())
words_set = set(open("allwords.txt").read().split())
words_set = set(item.lower() for item in words_set)

## Features

Unigrams for the data splitting on spaces

In [3]:
def get_unigrams():
    unigrams = Counter()
    for row_index, row in tweets.iterrows():
        s = row['tweet'].split()
        for word in s:
            unigrams[word] += 1
    return unigrams

### Load Emoji Dict


In [4]:
# loading the emoji dataset
def make_emoji_dict():
    emoji_dict = {}
    with open("emoji_image_to_whatIs.txt", 'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.strip().split('\t')
            emoji_dict[tokens[0]] = tokens[1]
    return emoji_dict
emoji_dict = make_emoji_dict()
#emoji_dict

In [5]:
#split into quartiles?
def find_quartile_values(counts):
    counts.sort()
    first = np.quantile(counts, .25)
    second = np.quantile(counts, .5)
    third = np.quantile(counts, .75)
    return [first, second, third] 

### Emoji Sentiment Features

Emoji sentiment dictionary

Features:
Negative Sentiment,
Neutral Sentiment,
Positive Sentiment,
Overall Sentiment

In [6]:
# loading the emoji dataset
def load_emoji_sentiment_dict():
    emoji_sentiment_dict = {}
    with open("emoji_image_sentimentScore_definition_category.txt", 'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.strip().split('\t')
            #each emoji has a list of 5 terms for a value
            #emoji_sentiment_dict[tokens[0]] = [('neg':tokens[1]), ('neutral':tokens[2]), ('positive':tokens[3]), ('sentiment score':tokens[4]), ('name':tokens[5]), ('category':tokens[6])]
            emoji_sentiment_dict[tokens[0]] = [tokens[1], tokens[2], tokens[3], tokens[4], tokens[5], tokens[6]]
    return emoji_sentiment_dict
emoji_sentiment_dict = load_emoji_sentiment_dict()


## Cite

Emoji	Neg[0...1]	Neut[0...1]	Pos[0...1]	Sentiment score[-1...+1]	Unicode name	Unicode block

@article{Kralj2015emojis,
  author={{Kralj Novak}, Petra and Smailovi{\'c}, Jasmina and Sluban, Borut and Mozeti\v{c}, Igor},
  title={Sentiment of emojis},
  journal={PLoS ONE},
  volume={10},
  number={12},
  pages={e0144296},
  url={http://dx.doi.org/10.1371/journal.pone.0144296},
  year={2015}
}


In [7]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

In [8]:
negativeValues = []
def make_emoji_negative_sentiment_bins():
    for row_index, row in tweets.iterrows():
        s = tknzr.tokenize(row['tweet'])
        score = 0
        for word in s:
            if word in emoji_sentiment_dict:
                #negative  
                score += float(emoji_sentiment_dict[word][0])
                negativeValues.append(score)
    return find_quartile_values(negativeValues)

In [9]:
# get emoji counts for data
def negative_sentiment_emojis(tweet):
    s = tknzr.tokenize(tweet)
    score = sum(map(lambda word : float(emoji_sentiment_dict[word][0]) if word in emoji_sentiment_dict else 0, s))
    return score

In [10]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
neutralValues = []

def make_emoji_neutral_sentiment_bins():
    for row_index, row in tweets.iterrows():
        s = tknzr.tokenize(row['tweet'])
        score = 0
        for word in s:
            if word in emoji_sentiment_dict:
                #neutral 
                score += float(emoji_sentiment_dict[word][1])
                neutralValues.append(score)
    return find_quartile_values(negativeValues)

In [11]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

# get emoji counts for data
def neutral_sentiment_emojis(tweet):
    s = tknzr.tokenize(tweet)
    score = sum(map(lambda word : float(emoji_sentiment_dict[word][1]) if word in emoji_sentiment_dict else 0, s))
    return score

In [12]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
positiveValues = []

def make_emoji_positive_sentiment_bins():
    tweets = train_data[['tweet']]
    for row_index, row in tweets.iterrows():
        s = tknzr.tokenize(row['tweet'])
        score = 0
        for word in s:
            if word in emoji_sentiment_dict:
                #positive 
                score += float(emoji_sentiment_dict[word][2])
                positiveValues.append(score)
    return find_quartile_values(positiveValues)
#make_emoji_positive_sentiment_bins()

In [13]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

# get emoji counts for data
def positive_sentiment_emojis(tweet):
    s = tknzr.tokenize(tweet)
    score = sum(map(lambda word : float(emoji_sentiment_dict[word][2]) if word in emoji_sentiment_dict else 0, s))
    return score

In [14]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

sentimentValues = []

def make_emoji_overall_sentiment_bins():
    for row_index, row in tweets.iterrows():
        s = tknzr.tokenize(row['tweet'])
        score = 0
        for word in s:
            if word in emoji_sentiment_dict:
                #sentiment
                score += float(emoji_sentiment_dict[word][3])
                sentimentValues.append(score)
    return find_quartile_values(sentimentValues)

In [15]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

# get emoji counts for data
def overall_sentiment_emojis(tweet):
    s = tknzr.tokenize(tweet)
    count = sum(map(lambda word : float(emoji_sentiment_dict[word][3]) if word in emoji_sentiment_dict else 0, s))
    return count

## Profanity and Racist Lexicon counts

Make count features binary by finding median values over entire dataset

In [16]:
#profanity_banned from trinker-lexicon-4c5e22b
profanity_banned = set(["anal", "anus", "arse", "ass", "balls", "ballsack", "bastard", 
"biatch", "bitch", "bloody", "blow job", "blowjob", "bollock", 
"bollok", "boner", "boob", "bugger", "bum", "butt", "buttplug", 
"clitoris", "cock", "coon", "crap", "cunt", "damn", "dick", "dildo", 
"dyke", "f u c k", "fag", "feck", "felching", "fellate", "fellatio", 
"flange", "fuck", "fudge packer", "fudgepacker", "God damn", 
"Goddamn", "hell", "homo", "jerk", "jizz", "knob end", "knobend", 
"labia", "lmao", "lmfao", "muff", "nigga", "nigger", "omg", "penis", 
"piss", "poop", "prick", "pube", "pussy", "queer", "s hit", "scrotum", 
"sex", "sh1t", "shit", "slut", "smegma", "spunk", "tit", "tosser", 
"turd", "twat", "vagina", "wank", "whore", "wtf"])

In [17]:
def make_profanity_bins():
    profanity = []
    for row_index,row in tweets.iterrows():
        s = row['tweet'].split()
        count = sum(map(lambda word : 1 if word in profanity_banned else 0, s))
        profanity.append(count)
    return find_quartile_values(profanity)
#make_profanity_bins()

In [50]:
# loading the emoji dataset
def load_racist_lexicon():
    racist_lexicon = []
    with open("racist_lexicon.txt", 'r') as f:
        lines = f.readlines()
        for line in lines:
            tokens = line.strip().split('\t')
            word = tokens[1].split("\"") 
            racist_lexicon.append(word[1])
    racist_lexicon = set(racist_lexicon)
    return racist_lexicon
racist_lexicon = load_racist_lexicon()

In [19]:
#NOTE  ------ to discuss ------
#very sparse, maybe this should be contains as if it's a word in that lexicon, 
#it's pretty darn offensive if not guaranteed to be hate speech

def make_racist_lexicon_bins():
    racist_words = []
    for row_index,row in tweets.iterrows():
        s = row['tweet'].split()
        count = sum(map(lambda word : 1 if word in racist_lexicon else 0, s))
        racist_words.append(count)
    return find_quartile_values(racist_words)
make_racist_lexicon_bins()

[0.0, 0.0, 0.0]

In [20]:
def make_at_bins():
    at_counts = []
    for row_index,row in tweets.iterrows():
        s = row['tweet'].split()
        count = sum(map(lambda word : 1 if '@' in word else 0, s))
        at_counts.append(count)
    return find_quartile_values(at_counts)
make_at_bins()

[0.0, 1.0, 1.0]

In [21]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
emoji_counts = []

# get emoji counts for data
def make_emoji_bins():
    for row_index,row in tweets.iterrows():
        s = tknzr.tokenize(row['tweet'])
        count = sum(map(lambda word : 1 if word in emoji_dict else 0, s))
        emoji_counts.append(count)
    return find_quartile_values(emoji_counts)

In [22]:
def make_num_token_bins():
    lens = []
    for row_index,row in tweets.iterrows():
        s = row['tweet'].split()
        count = len(s)
        lens.append(count)
    
    return find_quartile_values(lens)

In [23]:
def make_swear_bins():
    bad_words_counts = []
    for row_index,row in tweets.iterrows():
        s = row['tweet'].split()
        tot_bad = 0
        for word in s:
            word = word.replace(".","").replace(",","").replace("!","").replace("?","").replace(";","")
            if word.lower() in bad_words_set:
                tot_bad+=1
        bad_words_counts.append(tot_bad)
    
    return find_quartile_values(bad_words_counts)

In [24]:
def make_mention_bins():
    mentions = []
    for row_index,row in tweets.iterrows():
        s = row['tweet'].split()
        count = sum(map(lambda word : 1 if '@' in word else 0, s))
        mentions.append(count)
    return find_quartile_values(mentions)

In [25]:
def make_hashtag_bins():
    hashtag_counts = []
    at_sum = 0
    for row_index,row in tweets.iterrows():
        s = row['tweet'].split()
        count = sum(map(lambda word : 1 if '#' in word else 0, s))
        hashtag_counts.append(count)
    
    return find_quartile_values(hashtag_counts)

In [26]:
def make_misspelling_bins():
    misspell_counts = []
    for row_index,row in tweets.iterrows():
        s = row['tweet'].split()
        tot_misspelled = 0
        for word in s:
            word = word.replace(".","").replace(",","").replace("!","").replace("?","").replace(";","")
            if word.lower() not in words_set:
                tot_misspelled+=1
        misspell_counts.append(tot_misspelled)
    
    return find_quartile_values(misspell_counts)

In [28]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

# get emoji counts for data
def count_emojis(tweet):
    s = tknzr.tokenize(tweet)
    count = sum(map(lambda word : 1 if word in emoji_dict else 0, s))
    return count

In [29]:
#Get All Eng Vocabulary
import json 
import requests
import pandas as pd
from hatebase import HatebaseAPI
key = 'rTUCCDVYCcsEGmVKzQJjKwDFQsNcvUNa'
hatebase = HatebaseAPI({"key": key})
filters = {"language": "eng"}
format = "json"
# initialize list for all vocabulary entry dictionaries
eng_vocab = []
response = hatebase.getVocabulary(filters=filters, format=format)
pages = response["number_of_pages"]
# fill the vocabulary list with all entries of all pages
# this might take some time...
for page in range(1, pages+1):
    filters["page"] = str(page) 
    response = hatebase.getVocabulary(filters=filters, format=format)
    eng_vocab.append(response["result"])

In [30]:
df_eng_vocab = pd.DataFrame()
# fill df
for elem in eng_vocab:
    df_eng_vocab = df_eng_vocab.append(elem)
# reset the df index
df_eng_vocab.reset_index(drop=True, inplace=True)

In [31]:
#df_eng_vocab

In [32]:
hatebase_words_set = set(open("hatebase_terms.txt").read().split())

Get lists of all words from hatebase pertaining to a certain category

In [33]:
hb_religion = df_eng_vocab.loc[df_eng_vocab['is_about_religion']][['term']].values
hb_sexual_orientation = df_eng_vocab.loc[df_eng_vocab['is_about_sexual_orientation']][['term']].values
hb_ethnicity = df_eng_vocab.loc[df_eng_vocab['is_about_ethnicity']][['term']].values
hb_disability = df_eng_vocab.loc[df_eng_vocab['is_about_disability']][['term']].values
hb_social_class = df_eng_vocab.loc[df_eng_vocab['is_about_class']][['term']].values
hb_nationality = df_eng_vocab.loc[df_eng_vocab['is_about_nationality']][['term']].values
hb_gender = df_eng_vocab.loc[df_eng_vocab['is_about_gender']][['term']].values

Bins contain the splits for which bin a tweet's feature counts will land in

In [36]:
num_token_bins = make_num_token_bins()
swear_bins = make_swear_bins()
at_bins = make_at_bins()
hashtag_bins = make_hashtag_bins()
emoji_bins = make_emoji_bins()
#hatebase_words_bins = make_hatebase_bins()
misspell_bins =  make_misspelling_bins()
emoji_words_bins = make_emoji_bins()
emoji_overall_sentiment_bins = make_emoji_overall_sentiment_bins()
emoji_negative_sentiment_bins = make_emoji_negative_sentiment_bins()
emoji_positive_sentiment_bins = make_emoji_positive_sentiment_bins()
emoji_neutral_sentiment_bins = make_emoji_neutral_sentiment_bins()
profanity_bins = make_profanity_bins()
racist_bins = make_racist_lexicon_bins()

More binary features

In [37]:
def has_more_upper(tweet):
    total_caps = sum(map(lambda ch : 1 if ch.isupper() else 0, tweet))
    if total_caps > len(tweet) // 2:
        return 1
    return 0

In [38]:
def has_consecutive_punc(tweet):
    for word in tweet.split():
        if 'http://' in word: continue
        for i in range(len(word)-1):
            if word[i] in string.punctuation and word[i+1] in string.punctuation:
                return 1
    return 0

In [39]:
third_person = ['he', 'she', 'they', 'him', 'her', 'them', 'his', 'hers', 'their', 'theirs', 'themselves', 'himself', 'herself']
second_person = ['you', 'your', 'yours']
first_person =['i', 'me', 'my', 'mine', 'we', 'us', 'our', 'ours']

In [40]:
# Find if first word in tweet is a third person pronoun. Also check if number of third person pronouns is greater
# than first person pronouns
def get_pronouns(tweet):
    first_pronoun = 0
    third_and_second_greater_than_first = 0
    words = tweet.split()
    if words[0] in third_person or words[0] in second_person: 
        first_pronoun = 1
    
    first_person_count = 0
    third_second_person_count = 0
    for word in words:
        if word in third_person or word in second_person:
            third_second_person_count += 1
        elif word in first_person:
            first_person_count += 1
            
    if first_person_count < third_second_person_count:
        third_and_second_greater_than_first = 1
       
    return first_pronoun, third_and_second_greater_than_first

Cite lexicon:
    Minqing Hu and Bing Liu. "Mining and Summarizing Customer Reviews." 
        Proceedings of the ACM SIGKDD International Conference on Knowledge 
        Discovery and Data Mining (KDD-2004), Aug 22-25, 2004, Seattle, 
        Washington, USA,

In [41]:
pos_words = set(open("opinion-lexicon-English/positive-words.txt").read().split())
neg_words = set(open("opinion-lexicon-English/negative-words.txt").read().split())

In [42]:
#returns 1, 1 if negative words outnumber positive words and there are no positive words
def get_sentiment(tweet):
    pos_count = 0
    neg_count = 0
    for word in tweet.split():
        if word in pos_words:
            pos_count += 1
        if word in neg_words:
            neg_count += 1
    if neg_count > pos_count:
        if pos_count > 0:
            return 1, 0
        else:
            return 1, 1
    else:
        if pos_count > 0:
            return 0, 0
        else:
            return 0, 1

In [43]:
#returns one if a word in the tweet has non alphanumeric characters (not including punctuation at the end of a word)
def contains_non_alphanum(tweet):
    words = tweet.split()
    for word in words:
        if not word.isalnum():
            if word[-1] not in string.punctuation:
                return 1
    return 0

In [44]:
#Find most common unigrams
unigram_counts = get_unigrams()
top_unigrams = unigram_counts.most_common(100)

In [45]:
#each count feature has four bins: 1 for < 25th percentile, 2 for < 50th percentile, 
#3 for < 75th percentile, and 4 for < 100th percentile
def find_bin(count, bin_name):
    for i in range(len(bin_name)):
        if count < bin_name[i]:
            return i + 1
    return len(bin_name)

Process the tweets by getting their feature representations

In [54]:
def process_tweets(tweets, judgements):
#     for word in [u[0] for u in top_unigrams]:
#         tweets[word] = tweets['tweet'].str.contains(word).astype(int)
    word_counts = []
    swear_counts = []
    at_counts = []
    emoji_counts = []
    
    emoji_negative = []
    emoji_positive = []
    emoji_neutral = []
    emoji_overall_sentiment = []
    contains_emoji = []
    
    profanity_counts = []
    contains_profanity = []
    racist_counts = []
    contains_racist = []
    
    contains_at = []
    hashtag_counts = []
    contains_hashtag = []
    consecutive_punc = []
    more_upper = []
    first_pronoun = []
    fewer_first_person = []
    more_negative = []
    no_positive = []
    contains_url = []
    not_alphanum = []
    misspellings = []
    disagreements = []
    in_hatebase = []
    about_gender = []
    about_religion = []
    about_sexual_orientation = []
    about_ethnicity = []
    about_disability = []
    about_social_class = []
    about_nationality = []


    for tweet in tweets['tweet']:
        #count tokens
        tweet_words = tweet.split()
        num_token_bin = find_bin(len(tweet_words), num_token_bins)
        word_counts.append(num_token_bin)
        
        #emoji tokenizer
        s = tknzr.tokenize(tweet)
        #num_token_bin = find_bin(len(s), num_token_bins)
        #word_counts.append(num_token_bin)
        
        #count swear words, misspellings, and if a word is in hatebase
        misspell_count = 0
        tot_bad = 0
        tot_prof = 0
        tot_racist = 0
        hatebase = 0
        hatebase_words = []
        for word in tweet_words:                #Use regexs? 
            word = word.replace(".","").replace(",","").replace("!","").replace("?","")
            if word.lower() in profanity_banned:
                tot_prof+=1
                isProfane = 1
            if word.lower() in racist_lexicon:
                tot_racist+=1
                isRacist = 1
            if word.lower() in bad_words_set:
                tot_bad+=1
            if word.lower() not in words_set:
                misspell_count+=1
            if word.lower() in hatebase_words_set:
                hatebase = 1
                hatebase_words.append(word)
        swear_bin = find_bin(tot_bad, swear_bins)
        swear_counts.append(swear_bin)
        misspell_bin = find_bin(misspell_count, misspell_bins)
        misspellings.append(misspell_bin)
        in_hatebase.append(hatebase)
        
        racist_counts.append(tot_racist)
        racist_bin = find_bin(tot_racist, racist_bins)
        
        
        profanity_counts.append(tot_prof)
        profanity_bin = find_bin(tot_prof, profanity_bins)
        
    
        gender = religion = sexual_orientation = ethnicity = disability = social_class = nationality = 0
        if hatebase == 1:
            for word in hatebase_words:
                if word in hb_gender: gender = 1
                if word in hb_religion: religion = 1
                if word in hb_sexual_orientation: sexual_orientation = 1
                if word in hb_ethnicity: ethnicity = 1
                if word in hb_disability: disability = 1
                if word in hb_social_class: social_class = 1
                if word in hb_nationality: nationality = 1
        about_gender.append(gender)
        about_religion.append(religion)
        about_sexual_orientation.append(sexual_orientation)
        about_ethnicity.append(ethnicity)
        about_disability.append(disability)
        about_social_class.append(social_class)
        about_nationality.append(nationality)
        contains_racist.append(isRacist)
        contains_profanity.append(isProfane)
        
        #count mentions
        at_count = tweet.count('@')
        if at_count > 0:
            contains_at.append(1)
        else:
            contains_at.append(0)
        at_bin = find_bin(at_count, at_bins)
        at_counts.append(at_bin)
        
        #count hashtags
        hash_count = tweet.count('#')
        if hash_count > 0:
            contains_hashtag.append(1)
        else:
            contains_hashtag.append(0)
        hash_bin = find_bin(hash_count, hashtag_bins)
        hashtag_counts.append(hash_bin)
        
        #count emojis
        emoji_count = count_emojis(tweet)
        if emoji_count > 0:
            contains_emoji.append(1)
        else:
            contains_emoji.append(0)
        emoji_bin = find_bin(emoji_count, emoji_bins)
        emoji_counts.append(emoji_bin)
        
        #emoji positive sentiment 
        emoji_positive_score = positive_sentiment_emojis(tweet) 
        emoji_positive_bin = find_bin(emoji_positive_score, emoji_bins)
        emoji_positive.append(emoji_positive_bin)
        
        #emoji neutral sentiment 
        emoji_neutral_score = neutral_sentiment_emojis(tweet) 
        emoji_neutral_bin = find_bin(emoji_neutral_score, emoji_bins)
        emoji_neutral.append(emoji_neutral_bin)
        
        #emoji negative sentiment 
        emoji_negative_score = negative_sentiment_emojis(tweet) 
        emoji_negative_bin = find_bin(emoji_negative_score, emoji_bins)
        emoji_negative.append(emoji_negative_bin)
        
        #emoji overall sentiment 
        emoji_overall_score = overall_sentiment_emojis(tweet) 
        emoji_overall_bin = find_bin(emoji_overall_score, emoji_bins)
        emoji_overall_sentiment.append(emoji_overall_bin)
        
        more_upper.append(has_more_upper(tweet))
        consecutive_punc.append(has_consecutive_punc(tweet))
        first, more = get_pronouns(tweet)
        first_pronoun.append(first)
        fewer_first_person.append(more)
        
        more_neg, pos = get_sentiment(tweet)
        more_negative.append(more_neg)
        no_positive.append(pos)
        
        if 'http://' in tweet:
            contains_url.append(1)
        else:
            contains_url.append(0)
            
        not_alphanum.append(contains_non_alphanum(tweet))
        
        #See if there were disagreements about classification
        i = tweets.loc[tweets['tweet']==tweet].index[0]
        total_votes = judgements.at[i, 'count']
        if (judgements.at[i, 'hate_speech'] == total_votes) or (judgements.at[i, 'offensive_language'] == total_votes)\
            or (judgements.at[i, 'neither'] == total_votes):
            disagreements.append(0)
        else:
            disagreements.append(1)
        
    tweets['Word Counts'] = word_counts
    tweets['Swear Counts'] = swear_counts
    tweets['Profanity Counts'] = profanity_counts
    tweets['Contains Profanity'] = contains_profanity
    tweets['Racist Counts'] = racist_counts
    tweets['Contains Racist'] = contains_racist
    tweets['@ Counts'] = at_counts
    tweets['Mention'] = contains_at
    tweets['Contains Hashtag'] = contains_hashtag
    tweets['Hashtag Counts'] = hashtag_counts
    tweets['Emoji Counts'] = emoji_counts
    tweets['Contains Emoji'] = contains_emoji
    tweets['Negative Emoji'] = emoji_negative
    tweets['Positive Emoji'] = emoji_positive
    tweets['Neutral Emoji'] = emoji_neutral 
    tweets['Overall Emoji'] = emoji_overall_sentiment
    tweets['Consecutive Punctuation'] = consecutive_punc
    tweets['Majority Uppercase Letters'] = more_upper
    tweets['First Word Second or Third Person Pronoun'] = first_pronoun
    tweets['More Second or Third Person Pronouns than First'] = fewer_first_person
    tweets['Majority Negative Words'] = more_negative
    tweets['No Positive Words'] = no_positive
    tweets['Contains URL'] = contains_url
    tweets['Contains Non Alphanumeric Word'] = not_alphanum
    tweets['Misspelling Count'] = misspellings
    tweets['Judgement Disagreements'] = disagreements
    tweets['About Gender (Hatebase)'] = about_gender
    tweets['About Religion (Hatebase)'] = about_religion
    tweets['About Ethnicity (Hatebase)'] = about_ethnicity
    tweets['About Sexual Orientation (Hatebase)'] = about_sexual_orientation
    tweets['About Disability (Hatebase)'] = about_disability
    tweets['About Class (Hatebase)'] = about_social_class
    tweets['About Nationality (Hatebase)'] = about_nationality
    X = tweets[[col for col in tweets.columns if col!="tweet"]].values
    return X

Combine train and dev sets for k-fold cross validation

In [55]:
data = pd.concat([train_data, dev_data], sort=False)
tweets = data[['tweet']]
judgements = data[['count', 'hate_speech', 'offensive_language', 'neither']]

In [56]:
X = process_tweets(tweets, judgements)
y = data['class'].values

/Users/susanabenavidez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/susanabenavidez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:183: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/susanabenavidez/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://

In [57]:
X.shape

(22214, 33)

## Baseline Models

In [58]:
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
cv = KFold(n_splits=10, random_state=42, shuffle=False)

Run 10-fold cross validation on combined training and dev sets on LR, SVM, and NB models

In [59]:
LR = LogisticRegression(fit_intercept=True, max_iter=1000, solver='lbfgs', multi_class='ovr')
lr_y_pred = cross_val_predict(LR, X, y, cv=cv)

In [60]:
print("LR:", classification_report(y, lr_y_pred))

LR:              precision    recall  f1-score   support

          0       0.49      0.15      0.24      1282
          1       0.89      0.93      0.91     17186
          2       0.67      0.69      0.68      3746

avg / total       0.83      0.84      0.83     22214



In [61]:
svm = SVC(gamma='auto') 
svm_y_pred = cross_val_predict(svm, X, y, cv=cv)
print(classification_report(y, svm_y_pred))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1282
          1       0.89      0.93      0.91     17186
          2       0.66      0.77      0.71      3746

avg / total       0.80      0.85      0.83     22214



In [62]:
gnb = GaussianNB()
nb_y_pred = cross_val_predict(gnb, X, y, cv=cv)
print(classification_report(y, nb_y_pred))

             precision    recall  f1-score   support

          0       0.17      0.78      0.27      1282
          1       0.95      0.72      0.82     17186
          2       0.68      0.57      0.62      3746

avg / total       0.86      0.70      0.76     22214

